In [28]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
import pandas as pd
from rdkit import Chem
from tqdm import tqdm
from serenityff.charge.tree.dash_tree import DASHTree
from serenityff.charge.tree.atom_features import AtomFeatures
from rdkit import Chem
import torch
import sys
#add path to custom_featurization_stuff
sys.path.append("/localhome/cschiebroek/MDFPs/mdfptools/carl/Experiments_result_analysis/")
from custom_featurization_stuff import get_graph_from_mol
allowable_set= ["C","N","O","F","P","S","Cl","Br","I","H"]
from sklearn.model_selection import train_test_split
#also add r2
from sklearn.metrics import r2_score
#also add pearson
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
#start with loading the data
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools

# Function to read SDF file and extract properties
def read_sdf(file_path):
    suppl = Chem.SDMolSupplier(file_path)
    data = []
    for mol in suppl:
        if mol is not None:
            smiles = Chem.MolToSmiles(mol)
            logP = mol.GetProp('LogP') if mol.HasProp('LogP') else None
            logVP = mol.GetProp('LogVP') if mol.HasProp('LogVP') else None
            data.append({'SMILES': smiles, 'logP': logP, 'logVP': logVP})
    return data

# File paths for your SDF files
vp_path = 'OPERA_Data/VP_QR.sdf'
logp_path = 'OPERA_Data/LogP_QR.sdf'

# Read SDF files and extract data
data_vp = read_sdf(vp_path)
data_logp = read_sdf(logp_path)

# Create Pandas dataframes
df_vp = pd.DataFrame(data_vp)
df_logp = pd.DataFrame(data_logp)

#get the smiles for both
smiles_vp = df_vp['SMILES'].tolist()
smiles_logp = df_logp['SMILES'].tolist()
#get combined smiles
smiles_all = smiles_vp + smiles_logp
smiles_all = [s for s in smiles_all if s is not None]
smiles_all = list(set(smiles_all))
#foreach smile in overlap, get the logP from df2 and the logVP from df1
logP = []
logVP = []
for smile in smiles_all:
    #append val or 0.0 if not found
    logP.append(df_logp[df_logp['SMILES'] == smile]['logP'].values[0] if smile in smiles_logp else 0.0)
    logVP.append(df_vp[df_vp['SMILES'] == smile]['logVP'].values[0] if smile in smiles_vp else 0.0)
    
#make a new dataframe
df = pd.DataFrame({'SMILES': smiles_all, 'logP': logP, 'logVP': logVP})
df



[15:56:26] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 12 22 23 24 25 26 27
[15:56:26] ERROR: Could not sanitize molecule ending on line 38202
[15:56:26] ERROR: Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 12 22 23 24 25 26 27


,SMILES,logP,logVP
0,CCCCCCCC1CCCCC1,0.0,-1.3978998382903127
1,CC(C)(C)N(NC(=O)c1ccccc1)C(=O)c1ccccc1Cl,2.59,0.0
2,CC(=O)Nc1ccc(Cl)c(C(F)(F)F)c1,3.33,0.0
3,CCc1cc(OP(=S)(OC)OC)ccc1[N+](=O)[O-],3.74,0.0
4,NC(=O)c1cccc([N+](=O)[O-])c1,0.77,0.0
...,...,...,...
15075,CC(C)=CC1C(C(=O)OCc2cccc(I)c2)C1(C)C,6.61,0.0
15076,N#Cc1c[nH]cc1-c1cccc2c1OC(F)(F)O2,4.12,-8.533099771728077
15077,CN(C)CCOC(=O)c1ccc([N+](=O)[O-])cc1,1.7,0.0
15078,O=C(Br)c1ccccc1,0.0,-0.6536000773322567


In [ ]:
#do histograms of the logps and vps
logp_vals = df['logP'].tolist()
vp_vals = df['logVP'].tolist()
sns.set_theme(style="whitegrid")
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
sns.histplot(logp_vals, ax=ax[0])
ax[0].set_title('logP')
sns.histplot(vp_vals, ax=ax[1])
ax[1].set_title('logVP')
plt.show()


In [11]:
tree = DASHTree(tree_folder_path='/localhome/cschiebroek/other/serenityff-charge/tree')

Loading DASH tree data


In [12]:
all_mols = [Chem.MolFromSmiles(smi) for smi in df['SMILES']]
mols = [m for m in all_mols if m.GetNumAtoms() > 1]
error_mol = [m for m in all_mols if m.GetNumAtoms() <= 1]
mols_with_charges = []
error_mols_charges = []
for m in tqdm(mols):
    try:
        mol = Chem.AddHs(m, addCoords=True)
        charges = tree.get_molecules_partial_charges(mol,chg_std_key='std',chg_key='result')["charges"]
    except:
        error_mols_charges.append(m)
        continue
    for i,atom in enumerate(mol.GetAtoms()):
        atom.SetDoubleProp('charge',charges[i])
    mols_with_charges.append(mol)


100%|██████████| 15079/15079 [05:28<00:00, 45.88it/s]


In [13]:
indices_to_drop_size = [all_mols.index(m) for m in error_mol]
indices_to_drop_charges = [all_mols.index(m) for m in error_mols_charges]
indices_to_drop_total = list(set(indices_to_drop_size + indices_to_drop_charges))
print(len(indices_to_drop_total), len(indices_to_drop_size), len(indices_to_drop_charges))

143 1 142


In [15]:
if indices_to_drop_total:
    print('Caution! Mols dropped')
    df = df.drop(indices_to_drop_total)

In [18]:
#set values of second two columns as floats
df['logP'] = df['logP'].astype(float)
df['logVP'] = df['logVP'].astype(float)

In [22]:
ys = df.iloc[:, -2:].values
y = torch.tensor(ys, dtype=torch.float32)
y = y.unsqueeze(1)

In [25]:
y[0]

tensor([[ 0.0000, -1.3979]])

In [23]:
assert len(mols_with_charges) == len(y)
data = [get_graph_from_mol(mol,i, allowable_set,no_y=True) for i,mol in enumerate(mols_with_charges)]
for i in range(len(data)):
    data[i].y = y[i]

CustomData(x=[39, 24], edge_index=[2, 78], edge_attr=[78, 11], y=[39], batch=[39], molecule_charge=[1], smiles='[H]C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C1([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C1([H])[H]', sdf_idx=0)


In [26]:
#basics
import os
from math import sqrt
import pandas
import numpy as np
import random


#torch stuff
import torch
from torch.nn import functional as F
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_
from torch_geometric.nn.models import AttentiveFP
from torch_geometric.loader import DataLoader



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#set random seeds
torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(False)

def train_multi(train_loader, model, optimizer, device, outputs):
    total_loss = total_examples = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        weighted_loss = num_labels = 0
        for i in range(outputs):
            y_tmp = data.y[:, i]
            out_tmp = out[:, i]
            # Find indices where labels are available
            present_label_indices = torch.nonzero(y_tmp != 0.0).view(-1)
            num_labels += len(present_label_indices)

            if len(present_label_indices) > 0:
                # Extract only the available label indices
                out_tmp_present = torch.index_select(out_tmp, 0, present_label_indices)
                y_tmp_present = torch.index_select(y_tmp, 0, present_label_indices)

                # Calculate MSE loss only for available labels
                loss_tmp_present = F.mse_loss(out_tmp_present, y_tmp_present)
                if not torch.isnan(loss_tmp_present):
                    weighted_loss += loss_tmp_present * len(present_label_indices)

        weighted_loss = weighted_loss / num_labels
        weighted_loss.backward()

        total_loss += float(weighted_loss) * data.num_graphs
        total_examples += data.num_graphs

        # clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        del data

    return sqrt(total_loss / total_examples)


def validate_multi(val_loader, model, outputs):
    total_loss = total_examples = 0
    for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        weighted_loss = num_labels = 0
        for i in range(outputs):

            y_tmp = data.y[:, i]
            out_tmp = out[:, i]
            # Find indices where labels are available
            present_label_indices = torch.nonzero(y_tmp != 0.0).view(-1)
            num_labels += len(present_label_indices)

            if len(present_label_indices) > 0:
                # Extract only the available label indices
                out_tmp_present = torch.index_select(out_tmp, 0, present_label_indices)
                y_tmp_present = torch.index_select(y_tmp, 0, present_label_indices)

                # Calculate MSE loss only for available labels
                loss_tmp_present = F.mse_loss(out_tmp_present, y_tmp_present)
                if not torch.isnan(loss_tmp_present):
                    weighted_loss += loss_tmp_present * len(present_label_indices)

        weighted_loss = weighted_loss / num_labels

        total_loss += float(weighted_loss) * data.num_graphs
        total_examples += data.num_graphs
        del data

    return sqrt(total_loss / total_examples)

def train_and_validate_multi(model, train_loader, val_loader, optimizer, num_epochs, outputs, verbose=True):
    train_losses = []
    val_losses = []
    scheduler = ReduceLROnPlateau(optimizer, patience=2, factor=0.9,verbose=False)

    min_val_los = 1000
    for epoch in range(num_epochs):
        model.train()
        train_loss = train_multi(train_loader, model, optimizer, device, outputs)
        train_losses.append(train_loss)

        model.eval()
        val_loss = validate_multi(val_loader, model, outputs)
        val_losses.append(val_loss)
        scheduler.step(val_loss)

        if val_loss < min_val_los:
            min_val_los = val_loss
            counter = 0
            torch.save(model.state_dict(), 'test_model.pt')

        else:
            counter += 1
        if counter > 10:
            if verbose:
                print('early stopping')
            break
        if verbose:
            print(f"Epoch {epoch + 1}/{num_epochs}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

In [29]:

torch.cuda.empty_cache()
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)
#split train in train and val
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=0)
# #datloaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

model= AttentiveFP(in_channels=24, hidden_channels=200, out_channels=2,
                            edge_dim=11, num_layers=4, num_timesteps=2,
                            dropout=0.0).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=10**-3,
                        weight_decay=10**-4)
train_and_validate_multi(model, train_loader, val_loader, optimizer, num_epochs=100, outputs=2, verbose=True)

Epoch 1/100: Train Loss: 1.5453, Val Loss: 1.0815
Epoch 2/100: Train Loss: 1.0209, Val Loss: 0.9635
Epoch 3/100: Train Loss: 0.8893, Val Loss: 0.8559
Epoch 4/100: Train Loss: 0.8236, Val Loss: 0.8044
Epoch 5/100: Train Loss: 0.7990, Val Loss: 0.8357
Epoch 6/100: Train Loss: 0.7630, Val Loss: 0.7707
Epoch 7/100: Train Loss: 0.7258, Val Loss: 0.7408
Epoch 8/100: Train Loss: 0.6903, Val Loss: 0.7275
Epoch 9/100: Train Loss: 0.6593, Val Loss: 0.7081
Epoch 10/100: Train Loss: 0.6488, Val Loss: 0.7363
Epoch 11/100: Train Loss: 0.6417, Val Loss: 0.6921
Epoch 12/100: Train Loss: 0.6216, Val Loss: 0.7320
Epoch 13/100: Train Loss: 0.6229, Val Loss: 0.7218
Epoch 14/100: Train Loss: 0.5946, Val Loss: 0.6843
Epoch 15/100: Train Loss: 0.5785, Val Loss: 0.6894
Epoch 16/100: Train Loss: 0.5756, Val Loss: 0.6978
Epoch 17/100: Train Loss: 0.5671, Val Loss: 0.7185
Epoch 18/100: Train Loss: 0.5562, Val Loss: 0.6721
Epoch 19/100: Train Loss: 0.5526, Val Loss: 0.6648
Epoch 20/100: Train Loss: 0.5334, Val Lo

In [ ]:
from scipy.stats import spearmanr,kendalltau
def get_preds_per_task(val_loader, model, outputs):
    preds = tuple([[] for i in range(outputs)])
    ys = tuple([[] for i in range(outputs)])
    counter = 0
    for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        for i in range(outputs):
            y_tmp = data.y[:, i]
            out_tmp = out[:, i]

            # Find indices where labels are available
            present_label_indices = torch.nonzero(y_tmp != 0).view(-1)

            if len(present_label_indices) > 0:
                counter += len(present_label_indices)
                # Extract only the available label indices
                out_tmp_present = torch.index_select(out_tmp, 0, present_label_indices)
                y_tmp_present = torch.index_select(y_tmp, 0, present_label_indices)
                out_tmp_present = [float(i) for i in out_tmp_present]
                y_tmp_present = [float(i) for i in y_tmp_present]
                preds[i].extend(out_tmp_present)  
                ys[i].extend(y_tmp_present)
    print(counter)
    return preds,ys


def get_stats(out_list,y_list):
    kendall_tau = kendalltau(y_list, out_list)[0]
    rmse_overall = np.sqrt(np.mean((np.array(y_list) - np.array(out_list)) ** 2))
    mae_overall = np.mean(np.abs(np.array(y_list) - np.array(out_list)))
    within_03_overall = np.mean(np.abs(np.array(y_list) - np.array(out_list)) < 0.3)
    within_1_overall = np.mean(np.abs(np.array(y_list) - np.array(out_list)) < 1)

    return kendall_tau,rmse_overall,mae_overall,within_03_overall,within_1_overall



In [ ]:
#not plot the outputs in same way, but for each task (colored)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
preds,ys = get_preds_per_task(test_loader, model, 6)
#get the names of the columns
column_names = df.columns[-6:]

for i in range(2):
    kendall_tau,rmse_overall,mae_overall,within_03_overall,within_1_overall = get_stats(preds[i],ys[i])
    r2 = r2_score(ys[i],preds[i])
    pearson = pearsonr(ys[i],preds[i]).statistic
    print('Pearson:',pearson)
    print('R2:',r2)
    plt.figure(figsize=(10, 10))
    plt.scatter(ys[i], preds[i], alpha=0.5)
    min_val = min(min(ys[i]),min(preds[i]))
    max_val = max(max(ys[i]),max(preds[i]))
    #add diagonal, and lines at +- 0.3 and +- 1
    plt.plot([min_val-1,max_val+1],[min_val-1,max_val+1],color='black')
    plt.plot([min_val-1,max_val+1],[min_val-1+0.3,max_val+1+0.3],color='black',linestyle='--')
    plt.plot([min_val-1,max_val+1],[min_val-1-0.3,max_val+1-0.3],color='black',linestyle='--')
    plt.plot([min_val-1,max_val+1],[min_val-1+1,max_val+1+1],color='black',linestyle='--')
    plt.plot([min_val-1,max_val+1],[min_val-1-1,max_val+1-1],color='black',linestyle='--')
    plt.xlabel('True')
    plt.ylabel('Predicted')
    plt.title(column_names[i])
    print(f'kendall_tau: {kendall_tau}')
    print(f'rmse_overall: {rmse_overall}')
    print(f'mae_overall: {mae_overall}')
    print(f'within_03_overall: {within_03_overall}')
    print(f'within_1_overall: {within_1_overall}')
    plt.show()
